### Engineering

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
step 1: partition the weights
step 2: 

In [ ]:
scatter, reduce

In [2]:
import torch.distributed as dist

In [ ]:
dist.get_rank()

step 1: determine local rank in parallel group
step 2: if local rank is zero, then it is the fist

In [ ]:
partition(input)

In [ ]:
clock cycle 1: F(0, 0)
clock cycle 2: F(1, 0), F(0, 1)
clock cycle 3: F(2, 0), F(1, 1)
clock cycle 4: F(2, 1)

In [ ]:
step 1: determine the partition size
step 2: split weight
step 3: calculate the local output
step 4: calculate the global output

In [ ]:
scatter, reduceb

In [ ]:
tensor.unbind(dim=0)

In [ ]:
with torch.cuda.device(device):
    total = x.sum()

In [4]:
class Recompute(torch.autograd.Function):
    @staticmethod
    def forward(ctx, phony, recomputed, function, input):
        ctx.recomputed = recomputed
        ctx.function = function
        ctx.input = input
        
        return phony

    @staticmethod
    def backward(ctx, grad_input):
        function = ctx.function
        input = ctx.input
        input_leaf = input.detach().requires_grad(
            input.requires_grad
        )
        
        with torch.enable_grad():
            output = function(input_leaf)
        
        ctx.recomputed.append((output, input_leaf))
        return tuple([None, None, None, input_leaf.grad])

In [ ]:
step 1: fp16, fp32 of the weights
step 2: do forward pass and backward pass using fp16
step 3: cast the gradients from fp16 to fp32
step 4: update

In [ ]:
step 1: send metadata
step 2: send raw data
step 3: receive metadata, and construct a placeholder tensor
step 4: fill raw data

In [ ]:
step 1: mask targets
step 2: calcualte local predicted logits
step 3: calculate the global predicted logits
step 4: log(...)
step 5: calculate the loss log(...) - global_predicted_logitsbb

In [5]:
def gather_tensor(x):
    world_size = dist.get_world_size()
    
    if world_size == 1:
        return x
    
    xs = [torch.zeros_like(x) for _ in range(world_size)]
    dist.all_gather(xs, x)
    xs = torch.cat(xs)
    
    return xs

In [6]:
from contextlib import contextmanager

In [7]:
@contextmanager
def use_stream(stream):
    if not isinstance(stream, torch.cuda.Stream):
        yield
        return
    
    with torch.cuda.stream(stream):
        yield

In [9]:
class Checkpoint(torch.autograd.Function):
    @staticmethod
    def forward(ctx, phony, recomputed, function, input):
        ctx.recompute = recompute
        with torch.no_grad():
            output = function(input)
        return output, phony

    @staticmethod
    def backward(ctx, grad_input):
        output, input_leaf = ctx.recompute.pop()
        
        with torch.enable_grad():
            torch.autograd.backward(output, grad_input)
        
        return tuple([None, None, None, input_leaf.grad])

In [ ]:
shared memory, file system, message

In [ ]:
step 1: send ready
step 

In [11]:
import threading

In [ ]:
thread = threading.Thread(target=print_numbers)

In [ ]:
thread.start()

In [12]:
def run_worker():
    pass

In [ ]:
thread = threading.Thread(target=run_worker)

In [ ]:
(n-1)*m

In [13]:
import torch.distributed.rpc as rpc
from torch.distributed.rpc import RRef

In [ ]:
class Agent:
    def __init__(self):
        self.id = RRef(self)

In [ ]:
step 1: logit lens across accumulated residual stream
step 2: logit lens across decompsoed residual stream
step 3: logit less across 

In [ ]:
_, cache = model.run_with_cache(tokens)

In [14]:
from transformer_lens.utils import get_act_name

In [15]:
hook_name = get_act_name("pattern", 0, "attn")

In [16]:
hook_name

'blocks.0.attn.hook_pattern'

In [ ]:
attn_pattern = cache[hook_name]

In [ ]:
str_tokens = model.to_str_tokens(tokens)

In [ ]:
mlp_names = [get_act_name("mlp_out", layer_idx) for layer_idx in range(n_layers)]
attn_names = [get_act_name("attn_out", layer_idx) for layer_idx in range(n_heads)]

input_components = ["embed", "pos_embed"]

for mlp_name, attn_name in zip(mlp_names, attn_names):
    input_components.append([mlp_name, attn_name])

In [ ]:
_, cache = model.run_with_cache(tokens)

In [ ]:
pre_ln = cache["ln_final"] @ final_ln_coeffs.T

In [ ]:
step 1: get input
step 2: construct task
step 3: put the task into queue
step 4: wait output
step 5: put out

In [ ]:
step 1: raw_data + meta_data = package
step 2: invoke rpc along with the package
step 3: 

In [17]:
import threading

In [18]:
event = threading.Event()

In [19]:
def run_worker():
    print("waiting")
    event.wait()
    print("received")

In [ ]:
thread = threading.Thread(target=run_worker)

In [ ]:
pool watcher, worker thread, jobselection

In [ ]:
step 1: attn = l0h00(resid)
step 2: resid2 = resid + atn

In [ ]:
mlp_names = [get_act_name("mlp_out", layer_idx) for layer_idx in range(n_layers)]
attn_names = [get_act_name("attn_out", layer_idx) for layer_idx in range(n_heads)]

In [ ]:
hook_names = ["embed", "pos_embed"]

In [ ]:
for mlp_name, attn_name in zip(mlp_names, attn_names):
    hook_names.append([attn_name, mlp_name])

In [ ]:
_, cache = model.run_with_cache(tokens)

In [ ]:
final_ln = cache["ln_final"]
pre_final_ln = final_ln @ final_ln_coeffs.T

In [ ]:
W_U = model.W_U
target_dir = pre_ln_final @ (W_U[:, 0] - W_U[:, 1])

In [ ]:
for name in

In [ ]:
W_E = model.W_E
W_Q = model.W_Q[1, 4]
W_K = model.W_K[1, 4]
W_O = model.W_O[0, 7]
W_V = model.W_V[0, 7]

In [ ]:
K = W_E W_V @ W_O @ W_K

In [ ]:
step 1: logits = final_resid @ W_U
step 2: final_resid = embed + pos_embed + sum(12 heads)

In [ ]:
W_E = model.W_E
W_Q = model.W_Q[1, 4]
W_K = model.W_K[1, 4]
W_O = model.W_O[0, 7]
W_V = model.W_V[0, 7]

In [ ]:
K = W_E @ W_V @ W_O @ W_K
Q = W_E @ W_Q

In [ ]:
embed = model.embed
mlp0 = model.blocks[0].mlp
ln01 = model.blocks[1].ln2
unembed = model.unembed
ln_final = model.ln_final

In [ ]:
name_tokens = model.to_tokens(names)
text_embeddings = model.embed(name_tokens)

In [ ]:
resid_after_mlp0 = text_embeddings + mlp0(ln01(text_embeddings))

In [ ]:
def resid_to_logits(resid):
    return unembed(ln_final())

In [ ]:
tokens = model.to_tokens(text)

In [ ]:
_, cache = model.run_with_cache(tokens)

In [20]:
layer_idx, head_idx = 9, 9

In [ ]:
attn_probs = cache["attn", layer_idx][:, head_idx]

In [ ]:
attn_prob[:, 0, -1]

In [ ]:
step 1: attn_out0 = L0H00(pre_resid)
step 2: mid_resid = attn_out0 + pre_resid
step 3: mlp0 = MLP0(mid_resid)
step 4: resid1 = mid_resid + mlp0
step 5: mlp1 = MLP1(resid1)
step 6: resid2 = resid1 + mlp1

In [ ]:
A @ x @ W_OV @ W_OV

In [ ]:
W_in = model.W_in[5, 1393]

In [21]:
from einops import einsum, rearrange

In [ ]:
def compute_consine_similarity(weight, feature):
    weight /= weight.norm(dim=-1)
    feature /= feature.norm(dim=-1)
    
    return einsum(weight, feature, "")

In [ ]:
w_in_blank = compute_cosine_similarity(W_in, )

In [ ]:
embed = model.embed
mlp0 = model.blocks[0].mlp0
ln01 = model.blocks[0].ln2
unembed = model.unembed
ln_final = model.ln_final

In [22]:
def resid2logits(resid):
    return unembed(ln_final(resid))

In [ ]:
name_tokens = model.to_tokens(text)
text_embeddings = embed(name_tokens)

In [ ]:
resid_after_mlp0 = text_embeddings + mlp0(ln01(text_embeddings))

In [ ]:
for layer_idx in range(n_layers):
    for head_idx in range(n_heads):
        W_OV = model.W_V[layer_idx, head_idx] @ model.W_O[layer_idx, head_idx]
        output = resid_after_mlp0 @ W_OV
        logits = resid2logits(output)
        
        top_predictions = torch.topk(logits, dim=-1, k=5)
        percentage = (top_predictions == name_tokens).any(dim=-1).float().mean()

In [ ]:
L00, MLP0, MLP1, L2H20 

In [ ]:
MLP(Attn(x@W_E))@W_U

In [ ]:
step 1: logits = resid @ unembed
step 2: resid = embed + pos_embed + layer0 + layer1
step 3: logits = [embed + pos_embed + layer0 + layer1] @ unembed
step 4: logits = embed @ unembed + pos_embed @ unembed + layer0 @ unembed + layer1 @ unembed

In [ ]:
0.42 - 0.69, 2%, met

In [ ]:
text_similarities = text_embeddings @ text_embeddings.T
image_similarities = image_embeddings @ image_embeddings.T

In [ ]:
targets = F.softmax(
    (text_similarities + image_similarities) / (2*temperature)
)

In [ ]:
retina 

In [ ]:
training task: loss, objective func
base optimizer: sgd, architecture

In [ ]:
def convert_to_distribution(states):
    q_values = q_function(states)
    return q_values.exp() / q_values.sum(dim=-1)

In [31]:
min_workers = 7
max_workers = 10

In [32]:
num_working = 8

In [37]:
def is_new_needed():
    return min_workers > num_working and max_workers 

In [38]:
is_new_needed()

False